In [109]:
#pip install pandas

In [110]:
#pip install matplotlib

In [111]:
#pip install scipy

In [112]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [113]:
# STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
# STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  
#STARTINGCSV2025=pd.read_csv('NHL2526seasonplayerstats.csv')  
STARTINGCSV2025=pd.read_csv('NewPlayerStats2526.csv')  

STARTINGCSV=STARTINGCSV2025#pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameId'])
STARTINGCSV[STARTINGCSV['homeRoadFlag']==1]


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
12,8477935,9,{'default': 'S. Bennett'},C,0,0,0,-1,0,1,0,4,0.555556,19:15,0,26,0,1,Sam,Bennett,FLA,2025020001,1
13,8478421,10,{'default': 'A. Greer'},L,1,0,1,1,5,5,0,1,0.000000,07:19,1,13,0,0,A.J.,Greer,FLA,2025020001,1
14,8482713,11,{'default': 'M. Samoskevich'},R,0,2,2,1,0,3,0,0,0.000000,11:18,0,21,0,0,Mackie,Samoskevich,FLA,2025020001,1
15,8479981,12,{'default': 'J. Gadjovich'},L,0,1,1,1,0,4,0,1,0.000000,07:18,1,12,0,1,Jonah,Gadjovich,FLA,2025020001,1
16,8477933,13,{'default': 'S. Reinhart'},C,0,0,0,-1,0,1,0,3,0.333333,18:14,2,23,2,0,Sam,Reinhart,FLA,2025020001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,8480860,7,{'default': 'K. Bahl'},D,0,0,0,0,0,0,0,2,0.000000,20:42,1,30,2,1,Kevin,Bahl,CGY,2025020110,1
3992,8484768,19,{'default': 'Z. Parekh'},D,0,0,0,0,0,0,0,1,0.000000,12:42,0,20,2,1,Zayne,Parekh,CGY,2025020110,1
3993,8479402,24,{'default': 'J. Bean'},D,0,0,0,1,0,0,0,1,0.000000,19:41,2,29,1,0,Jake,Bean,CGY,2025020110,1
3994,8477810,44,{'default': 'J. Hanley'},D,0,0,0,0,0,0,0,3,0.000000,12:08,1,19,0,0,Joel,Hanley,CGY,2025020110,1


In [114]:
# #https://medium.com/@alexandre.tkint/send-pandas-data-frame-via-email-in-python-c346314321fa


# from email.mime.application import MIMEApplication
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# import smtplib
# def send_email(send_to, subject, df):
#     send_from = "dummy@gmail.com"
#     password = "your_password"
#     message = """\
#     <p><strong>This is a test email&nbsp;</strong></p>
#     <p><br></p>
#     <p><strong>Greetings&nbsp;</strong><br><strong>Alexandre&nbsp;    </strong></p>
#     """
# for receiver in send_to:
#         multipart = MIMEMultipart()
#         multipart["From"] = send_from
#         multipart["To"] = receiver
#         multipart["Subject"] = subject  
#         attachment = MIMEApplication(df.to_csv())
#         attachment["Content-Disposition"] = 'attachment; filename=" {}"'.format(f"{subject}.csv")
#         multipart.attach(attachment)
#         multipart.attach(MIMEText(message, "html"))
#         server = smtplib.SMTP("smtp.gmail.com", 587)
#         server.starttls()
#         server.login(multipart["From"], password)
#         server.sendmail(multipart["From"], multipart["To"], multipart.as_string())
#         server.quit()

In [115]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,New Jersey,Devils,NJD
1,Colorado,Avalanche,COL
2,Vegas,Golden Knights,VGK
3,Montréal,Canadiens,MTL
4,Winnipeg,Jets,WPG
5,Carolina,Hurricanes,CAR
6,Pittsburgh,Penguins,PIT
7,Washington,Capitals,WSH
8,Detroit,Red Wings,DET
9,Utah,Mammoth,UTA


In [116]:
STARTINGCSV.head()

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
0,8477987,8,{'default': 'R. Donato'},C,0,0,0,0,0,3,0,1,0.000000,16:00,0,23,0,0,Ryan,Donato,CHI,2025020001,0
1,8477450,16,{'default': 'J. Dickinson'},C,0,0,0,0,0,0,0,1,0.578947,14:14,0,25,0,0,Jason,Dickinson,CHI,2025020001,0
2,8473422,17,{'default': 'N. Foligno'},L,0,0,0,-1,5,2,0,0,0.625000,11:48,0,21,2,0,Nick,Foligno,CHI,2025020001,0
3,8483450,20,{'default': 'R. Greene'},C,0,0,0,-1,0,0,0,0,0.250000,08:17,0,15,0,0,Ryan,Greene,CHI,2025020001,0
4,8478043,24,{'default': 'S. Lafferty'},C,0,0,0,-1,0,1,0,0,0.000000,08:07,0,13,0,0,Sam,Lafferty,CHI,2025020001,0


In [117]:
playerStats=STARTINGCSV[['Team','playerId','firstName','lastName','gameId','homeRoadFlag','sog','toi','position']]
playerStats['toi']=round(pd.to_numeric(playerStats['toi'].str.slice(0,2).str.replace(':', '', regex=False)
                                 )+(pd.to_numeric(playerStats['toi'].str.slice(3, 5))/60),2) #min+sec in decimal base not base60
playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('Team').GameNumber.transform(np.max)
playerStats['HomeGames'] = playerStats.groupby(['playerId']).homeRoadFlag.transform(np.sum)
playerStats['AwayGames']=playerStats['GP']-playerStats['HomeGames']
playerStats['TotalShotsSzn']=playerStats.groupby(['playerId']).sog.transform(np.sum)
playerStats['SeasonAvgSOG']= round(playerStats['TotalShotsSzn']/playerStats['GP'],2)
playerStats['AvgShotsSznByLoc']=playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean)

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)
playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).sog.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)
playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).sog.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)

playerStats['avgshotshomevaway']=round(playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean),2)


playerStats[playerStats['homeRoadFlag']==1]


,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,position,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway
12,FLA,8477935,Sam,Bennett,2025020001,1,4,19.25,C,8,8,3,5,18,2.25,2.666667,1,3.0,1,1.8,2.67
13,FLA,8478421,A.J.,Greer,2025020001,1,1,7.32,L,8,8,3,5,4,0.50,0.666667,1,0.4,1,0.4,0.67
14,FLA,8482713,Mackie,Samoskevich,2025020001,1,0,11.30,R,8,8,3,5,23,2.88,2.333333,1,2.4,1,2.3,2.33
15,FLA,8479981,Jonah,Gadjovich,2025020001,1,1,7.30,L,8,8,3,5,4,0.50,0.666667,1,0.6,1,0.4,0.67
16,FLA,8477933,Sam,Reinhart,2025020001,1,3,18.23,C,8,8,3,5,18,2.25,3.000000,1,3.0,1,1.8,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,CGY,8480860,Kevin,Bahl,2025020110,1,2,20.70,D,1,9,4,5,6,0.67,0.750000,0,0.4,1,0.6,0.75
3992,CGY,8484768,Zayne,Parekh,2025020110,1,1,12.70,D,1,9,4,5,11,1.22,1.000000,0,1.4,1,1.1,1.00
3993,CGY,8479402,Jake,Bean,2025020110,1,1,19.68,D,1,9,2,7,4,0.44,1.000000,0,0.0,1,0.4,1.00
3994,CGY,8477810,Joel,Hanley,2025020110,1,3,12.13,D,1,9,4,5,4,0.44,0.750000,0,0.2,1,0.4,0.75


In [118]:
res = playerStats.groupby(['playerId'])["sog"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['sog'])&(res['sog']<res['High'])].drop(columns=['Low', 'High'])

#toi
res = playerStats.groupby(['playerId'])["toi"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=ShotsNoOutliers.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['toi'])&(res['toi']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['Avgtoi']=ShotsNoOutliers.groupby(['playerId']).toi.transform(np.mean)

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['sog'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).sog.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,position,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway,Avgtoi,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,CHI,8477987,Ryan,Donato,2025020001,0,1,16.00,C,7,7,4,3,17,2.43,2.000000,1,3.0,1,1.7,2.00,15.412500,4,4,1.75
1,CHI,8477450,Jason,Dickinson,2025020001,0,1,14.23,C,5,7,2,5,7,1.00,1.000000,1,1.2,1,0.7,1.00,15.440000,2,2,1.50
2,CHI,8482703,Colton,Dach,2025020001,0,1,13.48,C,7,7,4,3,9,1.29,1.333333,1,1.6,1,0.9,1.33,13.472500,4,4,1.25
4,CHI,8476882,Teuvo,Teravainen,2025020001,0,1,18.08,C,7,7,4,3,5,0.71,0.666667,1,1.0,1,0.5,0.67,18.080000,1,1,1.00
5,CHI,8483493,Frank,Nazar,2025020001,0,1,20.23,C,7,7,4,3,11,1.57,2.000000,1,1.6,1,1.1,2.00,19.350000,2,2,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,CGY,8484180,Samuel,Honzek,2025020110,1,1,9.65,L,1,9,4,5,12,1.33,1.250000,0,1.6,1,1.2,1.25,12.125000,1,4,2.00
1620,CGY,8479066,Ryan,Lomberg,2025020110,1,1,8.43,L,1,9,4,5,7,0.78,1.000000,0,0.4,1,0.7,1.00,9.596667,1,3,1.00
1622,CGY,8475172,Nazem,Kadri,2025020110,1,3,22.77,C,1,9,4,5,21,2.33,3.250000,0,2.4,1,2.1,3.25,20.485000,1,4,3.00
1623,MTL,8483457,Lane,Hutson,2025020110,0,1,23.38,D,1,8,4,4,11,1.38,1.250000,0,1.6,1,1.1,1.25,23.057500,1,4,1.25


In [119]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,CHI,TBL
1,ANA,BOS
2,PHI,OTT
3,PIT,FLA
4,DET,NYI
5,SJS,NYR
6,UTA,STL
7,VAN,NSH
8,SEA,WPG
9,LAK,DAL


In [120]:
shotSummary=ShotsNoOutliers[['Team', 'playerId','firstName', 'lastName', 'Avgtoi','position','SeasonAvgSOG', 'LastFiveAvgShots',
'LastTenAvgShots', 'NoOutliersAvgSOG','avgshotshomevaway']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['Team'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='Team', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])
todaysSkaters['HomeFlag'] = np.where(todaysSkaters['Team'].isin(TodaysGames['HomeTeam'].tolist()), 1, 0)

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,Team,playerId,firstName,lastName,Avgtoi,position,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,avgshotshomevaway,Opponent,HomeFlag
0,ANA,8483445,Cutter,Gauthier,18.805000,L,4.67,5.6,2.8,5.500000,4.00,BOS,0
1,ANA,8483445,Cutter,Gauthier,18.805000,L,4.67,5.6,2.8,5.500000,5.00,BOS,0
2,SEA,8483524,Shane,Wright,16.070000,C,1.57,2.2,1.1,5.000000,2.20,WPG,0
3,CAR,8482093,Seth,Jarvis,21.036667,C,4.00,4.8,2.4,4.666667,4.00,COL,0
4,COL,8477492,Nathan,MacKinnon,21.525000,C,4.71,5.0,3.3,4.500000,5.33,CAR,1
5,COL,8477492,Nathan,MacKinnon,21.525000,C,4.71,5.0,3.3,4.500000,4.25,CAR,1
6,CAR,8482809,Jackson,Blake,16.133333,R,3.83,4.6,2.3,4.000000,5.50,COL,0
7,NSH,8476539,Jonathan,Marchessault,17.500000,R,2.14,2.6,1.5,4.000000,3.00,VAN,1
8,CAR,8482809,Jackson,Blake,16.133333,R,3.83,4.6,2.3,4.000000,3.00,COL,0
9,DAL,8478449,Roope,Hintz,19.800000,C,3.17,3.8,1.9,4.000000,3.33,LAK,1


In [121]:

goalieSTARTINGCSV2025=pd.read_csv('NewGoalieStats2526.csv')  
goalieSTARTINGCSV=goalieSTARTINGCSV2025
goalieSTARTINGCSV.sort_values(by=['gameId'])

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,firstName,lastName,Team,gameId,homeRoadFlag
0,8481519,30,{'default': 'S. Knight'},G,29/31,4/5,1/1,34/37,0.918919,2,1,0,0,3,59:04,True,L,37,34,Spencer,Knight,CHI,2025020001,0
1,8475683,72,"{'default': 'S. Bobrovsky', 'cs': 'S. Bobrovsk...",G,15/17,2/2,0/0,17/19,0.894737,2,0,0,0,2,60:00,True,W,19,17,Sergei,Bobrovsky,FLA,2025020001,1
2,8481668,37,"{'default': 'A. Silovs', 'cs': 'A. Šilovs', 's...",G,22/22,3/3,0/0,25/25,1.000000,0,0,0,0,0,60:00,True,W,25,25,Arturs,Silovs,PIT,2025020002,0
3,8478048,31,"{'default': 'I. Shesterkin', 'cs': 'I. Šesťork...",G,28/29,0/0,0/0,28/29,0.965517,1,0,0,0,1,59:37,True,L,29,28,Igor,Shesterkin,NYR,2025020002,1
4,8475809,41,{'default': 'S. Wedgewood'},G,17/17,7/8,0/0,24/25,0.960000,0,1,0,0,1,59:53,True,W,25,24,Scott,Wedgewood,COL,2025020003,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,8482076,50,{'default': 'N. Daws'},G,25/26,5/5,0/0,30/31,0.967742,1,0,0,0,1,60:00,True,W,31,30,Nico,Daws,NJD,2025020108,1
218,8476434,36,{'default': 'J. Gibson'},G,17/20,8/9,2/2,27/31,0.870968,3,1,0,0,4,56:49,True,L,31,27,John,Gibson,DET,2025020109,0
219,8481551,92,{'default': 'C. Ellis'},G,28/30,1/1,0/0,29/31,0.935484,2,0,0,0,2,59:54,True,W,31,29,Colten,Ellis,BUF,2025020109,1
220,8482487,75,{'default': 'J. Dobes'},G,31/32,3/3,2/2,36/37,0.972973,1,0,0,2,1,61:00,True,W,37,36,Jakub,Dobes,MTL,2025020110,0


In [122]:
goalieStats=goalieSTARTINGCSV[['Team','playerId','firstName','lastName','gameId','starter','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['PlayerGameNumber'] = goalieStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
goalieStats['TeamGameNumber'] = goalieStats.groupby(['Team'])['gameId'].cumcount(ascending=False)+1
goalieStats['PlayerGP']=goalieStats.groupby(['playerId']).starter.transform(np.sum)
goalieStats['TeamGP']=goalieStats.groupby(['Team']).starter.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['Team']).shotsAgainst.transform(np.sum)/goalieStats['TeamGP']
goalieStats['RankAvgShotsAgainst']=round(goalieStats['AvgShotsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgGoalsAgainst']=round(goalieStats['AvgGoalsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgSavePctg']=round(goalieStats['AvgSavePctg'].rank(pct=True)*100,2)

goalieStats['LastFiveFlag'] = np.where(goalieStats['TeamGameNumber'] >= goalieStats['TeamGP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['Team','PlayerGP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['Team'])['PlayerGP'].rank(ascending=False).astype(int)
summaryGoalieStats=summaryGoalieStats[['Team','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','Team':'GoalieteamAbbrev'})
summaryGoalieStats.head()

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,CHI,1,Spencer,Knight,87.39,22.07,93.24
1,FLA,1,Sergei,Bobrovsky,29.50,31.31,63.29
2,PIT,1,Arturs,Silovs,66.22,38.96,71.85
3,NYR,1,Igor,Shesterkin,33.11,2.03,98.42
4,COL,1,Scott,Wedgewood,24.55,12.61,82.88


In [142]:
full=todaysSkaters.merge(summaryGoalieStats
                         [summaryGoalieStats['GPRank']==1]
                         , left_on='Opponent', right_on='GoalieteamAbbrev', how='left'
                         ).drop(columns=['GoalieteamAbbrev','GPRank'])
full['lastfivemargin']=full['LastFiveAvgShots']-full['NoOutliersAvgSOG']

full['locationmargin']=full['avgshotshomevaway']-full['NoOutliersAvgSOG']
full['heatermargin']=full['LastFiveAvgShots']-full['LastTenAvgShots']
full['fixedmargin']=full['lastfivemargin']-full['heatermargin']
full['goalieweight']=2*((full['RankAvgShotsAgainst']-50)/100)
full['combinedmargin']=full['locationmargin']+full['fixedmargin']+full['goalieweight']

full['A_Standardized'] = (full['combinedmargin'] - full['combinedmargin'].mean()) / (full['combinedmargin'].std())

full['overrated'] = np.where(full['A_Standardized']<0,full['NoOutliersAvgSOG']-abs(full['A_Standardized']),
abs(full['A_Standardized'])-full['NoOutliersAvgSOG'])


full=full[['lastName','firstName', 'NoOutliersAvgSOG','combinedmargin','A_Standardized','overrated','fixedmargin','goalieweight',
'Avgtoi','position','Team', 'HomeFlag','Opponent','heatermargin',  'LastFiveAvgShots', 'LastTenAvgShots',
'lastfivemargin','avgshotshomevaway','locationmargin','RankAvgShotsAgainst'
]].drop_duplicates().rename(columns={"NoOutliersAvgSOG": "ExpectedSOG",
        "avgshotshomevaway": "avgshotbyLoc"}).round(2)


todaysdate=str(pd.Timestamp.now().date())
full.to_excel(todaysdate+'skaterstats.xlsx', index=False, engine='xlsxwriter')

In [143]:
#full.insert(loc=1, column='new_col', value=[str('test') for i in range(0,len(full))])

full.sort_values(by=['overrated']
    #by=['ExpectedSOG']
                        , ascending=True #False #or True for players trending down
                       ).reset_index(drop=True).head(25)

,lastName,firstName,ExpectedSOG,combinedmargin,A_Standardized,overrated,fixedmargin,goalieweight,Avgtoi,position,Team,HomeFlag,Opponent,heatermargin,LastFiveAvgShots,LastTenAvgShots,lastfivemargin,avgshotbyLoc,locationmargin,RankAvgShotsAgainst
0,Horvat,Bo,3.50,-0.41,0.14,-3.36,-0.90,-0.51,20.28,C,NYI,1,DET,2.6,5.2,2.6,1.70,4.50,1.00,24.55
1,Schmaltz,Nick,3.67,-0.09,0.43,-3.24,-0.37,-0.65,19.22,C,UTA,0,STL,2.3,5.6,3.3,1.93,4.60,0.93,17.34
2,DeBrincat,Alex,3.25,-0.36,0.18,-3.07,-0.95,0.64,17.85,R,DET,0,NYI,1.7,4.0,2.3,0.75,3.20,-0.05,81.76
3,Pastrnak,David,3.80,0.30,0.78,-3.02,-0.70,0.05,21.31,R,BOS,1,ANA,1.3,4.4,3.1,0.60,4.75,0.95,52.48
4,Garland,Conor,3.00,-0.51,0.05,-2.95,-0.90,0.39,21.04,R,VAN,0,NSH,1.7,3.8,2.1,0.80,3.00,0.00,69.37
5,Ehlers,Nikolaj,3.00,-0.51,0.06,-2.94,-1.00,-0.51,17.67,L,CAR,0,COL,2.0,4.0,2.0,1.00,4.00,1.00,24.55
6,Robertson,Jason,3.67,0.26,0.74,-2.93,-1.07,-0.01,19.74,L,DAL,1,LAK,2.6,5.2,2.6,1.53,5.00,1.33,49.55
7,Kempe,Adrian,3.50,0.20,0.69,-2.81,-1.10,0.55,19.22,R,LAK,0,DAL,2.0,4.4,2.4,0.90,4.25,0.75,77.48
8,DeBrincat,Alex,3.25,-0.06,0.45,-2.80,-0.95,0.64,17.85,R,DET,0,NYI,1.7,4.0,2.3,0.75,3.50,0.25,81.76
9,Zibanejad,Mika,3.25,0.08,0.58,-2.67,-0.15,0.98,21.20,C,NYR,1,SJS,1.7,4.8,3.1,1.55,2.50,-0.75,98.87


In [125]:
# pip install xlsxwriter

In [126]:
# margin=full
# margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']


# margin=margin[['teamAbbrev', 'HomeFlag', 'firstName', 'lastName', 'ExpectedSOG','AvgShotsSznByLoc',
#        'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
#         'GoalieLastName',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp',
#         'SeasonAvgSOG', 'playerId', 
#        'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

# margin.sort_values(by=['lastfivemargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [127]:
# margin['locationmargin']=margin['AvgShotsSznByLoc']-margin['ExpectedSOG']

# margin.sort_values(by=['locationmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)


In [128]:
# margin['combinedmargin']=margin['lastfivemargin']+margin['locationmargin']

# margin[['firstName',	'lastName','teamAbbrev',	'HomeFlag','combinedmargin',
# 		'ExpectedSOG','LastFiveAvgShots',	'AvgShotsSznByLoc'	,
#         	'LastTenAvgShots',	'Opponent',	'RankAvgShotsAgainst',	'GoalieLastName',
#             			'SeasonAvgSOG']].sort_values(by=['combinedmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [129]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df